In [1]:
import random, pprint, json, pprint
from collections import defaultdict
import numpy as np
from run import SphersVoronoi
from convert_la import convert_la, calculate_center

In [2]:
def next_seeds(n, voronoi_data):
    # voronoi_data: 二维数组，上一次迭代的结果
    res = [] # 用于存储每个种子点的坐标
    voronoi_regin = defaultdict(list)
    for i in range(len(voronoi_data)):
        for j in range(len(voronoi_data[0])):
            if voronoi_data[i][j] != 0:
                voronoi_regin[voronoi_data[i][j]].append(convert_la(n, [i, j]))
    for key, value in voronoi_regin.items():
        res.append(calculate_center(value))
    return res

def cesium_paint(n, data, colors):
    # 此函数用于将data数据更改为cesium可用的格式，data为positive_reverse()函数的返回值
    res = []
    for i in range(len(data) - 1):
        for j in range(len(data[0]) - 1):
            left_top = convert_la(n, [i, j])
            right_top = convert_la(n, [i, j + 1])
            left_bottom = convert_la(n, [i + 1, j])
            right_bottom = convert_la(n, [i + 1, j + 1])
            res.append([left_top + right_top + right_bottom + left_bottom, colors[data[i][j]]])
    return res

def find_polygons(grid):
    # Function to check if a cell is part of the boundary of a polygon
    def is_boundary(x, y, value):
        # Check boundaries of the grid
        if x < 0 or y < 0 or x >= len(grid) or y >= len(grid[0]):
            return True
        # Check if neighboring cell is different
        return grid[x][y] != value

    # Dictionary to store the vertices of each polygon
    polygons = {}

    # Iterate over the grid
    for i in range(1, len(grid)-1):
        for j in range(1, len(grid[i])-1):
            value = grid[i][j]
            # If the cell is a boundary, add it to the polygon's list of vertices
            direction = [[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1], [1, 0], [1, 1]]
            dir_tmp = [is_boundary(item[0], item[1], value) for item in direction]
            if sum(dir_tmp) > 2:
                if value not in polygons:
                    polygons[value] = []
                polygons[value].append((i, j))
    return polygons

In [3]:
n = 4 # 层数
size = 2 ** n + 1 # 边长
seed_num = 9 # 种子点数量
step = 2 # 质心迭代次数
colors = [[0, 0, 0]] + [[random.randrange(99, 206) for _ in range(3)] for _ in range(seed_num)] # 随机颜色列表
seed_list = [convert_la(n, [random.randrange(size), random.randrange(size)]) for _ in range(seed_num)] # 种子点列表

for i in range(step):
    print('第{}次迭代'.format(i+1))
    sv = SphersVoronoi(n, seed_list)
    sv.deal()
    data = sv.positive_reverse()
    # 保存可被Cesium使用的json文件
    # with open(f'./data/{n}-{seed_num}-{i+1}.json', mode='w', encoding='utf-8') as f:
    #     f.write(str(json.dumps(cesium_paint(n, data, colors))))
    # 保存为png图片
    # sv.paint(data, 'positive_reverse_sphere_{}.png'.format(i+1), colors)
    seed_list = next_seeds(n, data)

第1次迭代
第2次迭代


In [4]:
polygons = find_polygons(data)

In [5]:
res_dic = {}

for key, value in polygons.items():
    for item in value:
        la_coord = convert_la(n, item)
        if key not in res_dic:
            res_dic[key] = []
        res_dic[key].append(la_coord)

res_dic

{4: [[-135.0, -67.5],
  [-153.434948822922, -56.25],
  [-161.565051177078, -45.0],
  [-165.96375653207352, -33.75000000000001],
  [170.5376777919744, -11.25000000000002],
  [168.6900675259798, -22.50000000000001],
  [158.1985905136482, -11.25],
  [165.96375653207352, -33.75000000000001],
  [153.434948822922, -22.5],
  [143.13010235415598, -11.25],
  [161.565051177078, -45.0],
  [146.30993247402023, -33.75000000000001],
  [135.0, -22.50000000000001],
  [153.434948822922, -56.25],
  [135.0, -45.0],
  [123.6900675259798, -33.75000000000001],
  [135.0, -67.5],
  [116.56505117707799, -56.25],
  [108.43494882292201, -45.0]],
 1: [[-116.56505117707799, -56.25],
  [-71.56505117707799, -45.0],
  [-63.43494882292201, -56.25],
  [-45.00000000000001, -67.5],
  [71.56505117707799, -45.0],
  [63.43494882292201, -56.25],
  [45.00000000000001, -67.5]],
 2: [[-108.43494882292201, -45.0],
  [-104.03624346792648, -33.75000000000001],
  [-101.30993247402021, -22.50000000000001],
  [-99.46232220802563, -11

In [6]:
from sort_algorithm import sort_coordinates_clockwise

In [10]:
sort_tmp = []

for coor in sort_coordinates_clockwise(res_dic[1]):
    sort_tmp+=coor
print(sort_tmp)

[-45.00000000000001, -67.5, 45.00000000000001, -67.5, 63.43494882292201, -56.25, 71.56505117707799, -45.0, -71.56505117707799, -45.0, -63.43494882292201, -56.25, -116.56505117707799, -56.25]


In [38]:
for key, value in res_dic.items():
    res_dic[key] = sort_coordinates_clockwise(value)
    tmp = []
    for coor in res_dic[key]:
        tmp += coor
    res_dic[key] = tmp
res_dic

{1: [-135.0,
  -67.5,
  -45.00000000000001,
  -67.5,
  135.0,
  -67.5,
  45.00000000000001,
  -67.5,
  153.434948822922,
  -56.25,
  116.56505117707799,
  -56.25,
  63.43494882292201,
  -56.25,
  26.565051177077994,
  -56.25,
  -26.565051177077994,
  -56.25,
  71.56505117707799,
  -45.0,
  45.00000000000001,
  -45.0,
  18.434948822922017,
  -45.0,
  -18.434948822922017,
  -45.0,
  -153.434948822922,
  -56.25],
 2: [-116.56505117707799,
  -56.25,
  -63.43494882292201,
  -56.25,
  -45.00000000000001,
  -45.0,
  -71.56505117707799,
  -45.0,
  -33.690067525979785,
  -33.75000000000001,
  -56.309932474020215,
  -33.75000000000001,
  -45.0,
  -22.50000000000001,
  -75.96375653207353,
  -33.75000000000001,
  -36.86989764584401,
  -11.25,
  -63.43494882292201,
  -22.5,
  -53.13010235415599,
  -11.25,
  -45.00000000000001,
  0.0,
  -78.69006752597979,
  -22.50000000000001,
  -68.19859051364818,
  -11.25,
  -59.03624346792648,
  0.0,
  -53.13010235415599,
  11.25,
  -45.0,
  22.50000000000001,
 

In [39]:
res = []

for key, value in res_dic.items():
    res.append([value, colors[key]])

with open(f'./data/test-{n}-{seed_num}-{step}.json', mode='w', encoding='utf-8') as f:
    f.write(str(json.dumps(res)))